In [21]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import re
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from nltk import word_tokenize
from collections import Counter 

C:\Users\nr17754\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
tag_list = train['tags'].tolist()
li_flat = [] 
for i in tag_list:
    li_flat.append(str(i).split("|"))

In [4]:
flat_tags = [i for sublist in li_flat for i in sublist]
top_tags = Counter(flat_tags).most_common(500)
top_flat = []
for i in top_tags:
    top_flat.append(i[0])

In [5]:
train.shape

(786979, 4)

In [6]:
test.shape

(194323, 3)

In [94]:
x = (preprocessing(train['article'][0]))

In [95]:
x

'way sort requests flash application send server trying information client sends server chrome inspect element shows going on sure communication going on website interesting time sends server opponent end game sends game server sure till now images uploaded game thanks help p s trying use wireshark capture packets way communication doing pinging chesscube com realize ip address     listening packages ip address ip addr     thing lot tcp http packages http packages sending png images avatars users chat there people constantly speaking that understand going ip address idea it problem watch traffic net computer it know limit it'

In [82]:
(preprocessing(train['title'][0]))

'check requests server application written flash'

In [99]:
import nltk
x = remove_urls(train['article'][0])
x = cleanhtml(x)
tokens = word_tokenize(x)
tagged = nltk.pos_tag(tokens)
tagged

[('Is', 'VBZ'),
 ('there', 'EX'),
 ('a', 'DT'),
 ('way', 'NN'),
 ('to', 'TO'),
 ('find', 'VB'),
 ('what', 'WP'),
 ('sort', 'NN'),
 ('of', 'IN'),
 ('requests', 'NNS'),
 ('does', 'VBZ'),
 ('a', 'DT'),
 ('Flash', 'NNP'),
 ('application', 'NN'),
 ('send', 'NN'),
 ('to', 'TO'),
 ('a', 'DT'),
 ('server', 'NN'),
 ('?', '.'),
 ('I', 'PRP'),
 ('was', 'VBD'),
 ('trying', 'VBG'),
 ('to', 'TO'),
 ('see', 'VB'),
 ('what', 'WP'),
 ('information', 'NN'),
 ('a', 'DT'),
 ('client', 'NN'),
 ('sends', 'VBZ'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('server', 'NN'),
 ('using', 'VBG'),
 ('Chrome', 'NNP'),
 ('``', '``'),
 ('inspect', 'JJ'),
 ('element', 'NN'),
 ("''", "''"),
 (',', ','),
 ('but', 'CC'),
 ('it', 'PRP'),
 ('shows', 'VBZ'),
 ('me', 'PRP'),
 ('that', 'IN'),
 ('nothing', 'NN'),
 ('is', 'VBZ'),
 ('going', 'VBG'),
 ('on', 'IN'),
 ('.', '.'),
 ('But', 'CC'),
 ('for', 'IN'),
 ('sure', 'JJ'),
 ('the', 'DT'),
 ('communication', 'NN'),
 ('is', 'VBZ'),
 ('going', 'VBG'),
 ('on', 'IN'),
 ('.', '.'),
 ('The', 'D

In [111]:
## Pre-Processing Script 

# Function to remove html tags
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

# Function to remove URLs starting with https/http 
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

# POS TAG removal
def pos_remover(x):
    tokens = word_tokenize(x)
    tagged = nltk.pos_tag(tokens)
    pos_removed_sentence = ''
    for i in tagged:
        if ((i[1]=='IN')|(i[1]=='VBZ')|(i[1]=='WP')|(i[1]=='DT')|(i[1]=='TO')|(i[1]=='PRP')|(i[1]=='VB')|(i[1]=='VBG')|(i[1]=='CC')|(i[1]=='VB')|(i[1]=='MD')):
            continue
        else:
            pos_removed_sentence = pos_removed_sentence +" "+i[0]
    return pos_removed_sentence 

# Create a unique list without repetition of words 
def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ulist
a = ' '.join(unique_list(a.split()))

In [115]:
def preprocessing(x):
    x = remove_urls(x)
    x = cleanhtml(x)
    tokens = word_tokenize(str(x))
    tagged = nltk.pos_tag(tokens)
    x = pos_remover(tagged)
    x = str(x).lower()
    x = remove_stopwords(x)
    x = strip_punctuation(x)
    x = strip_multiple_whitespaces(x)
    x = ' '.join([re.sub(r'\d+', '',i) for i in word_tokenize(x)])
    x = ' '.join(unique_list(x.split()))
    return x

In [117]:
x = remove_urls(train['article'][0])

In [118]:
x

'<p>Is there a way to find what sort of requests does a Flash application send to a server?</p>\r\r\r\r\n\r\r\r\r\n<p>I was trying to see what information a client sends to the server using Chrome "inspect element", but it shows me that nothing is going on.</p>\r\r\r\r\n\r\r\r\r\n<p>But for sure the communication is going on. The website I am interesting in is <a href="/" rel="nofollow">/</a> and every time you make a move it somehow sends it to a server or may be just to another opponent. In the end of the game - it sends the game to the server for sure. But up till now, all I can see is just few images, being uploaded during the game.</p>\r\r\r\r\n\r\r\r\r\n<p>Thanks for help.</p>\r\r\r\r\n\r\r\r\r\n<p>P.S. I was trying to use wireshark to capture packets and in such a way to see the communication. Here what I was doing:</p>\r\r\r\r\n\r\r\r\r\n<p>Pinging chesscube.com to realize its ip-address: 78.47.2.115\r\r\r\r\nthan I am listening only for a packages from that ip address ip.addr 

In [119]:
train['article'][0]

'<p>Is there a way to find what sort of requests does a Flash application send to a server?</p>\r\r\r\r\n\r\r\r\r\n<p>I was trying to see what information a client sends to the server using Chrome "inspect element", but it shows me that nothing is going on.</p>\r\r\r\r\n\r\r\r\r\n<p>But for sure the communication is going on. The website I am interesting in is <a href="http://www.chesscube.com/" rel="nofollow">http://www.chesscube.com/</a> and every time you make a move it somehow sends it to a server or may be just to another opponent. In the end of the game - it sends the game to the server for sure. But up till now, all I can see is just few images, being uploaded during the game.</p>\r\r\r\r\n\r\r\r\r\n<p>Thanks for help.</p>\r\r\r\r\n\r\r\r\r\n<p>P.S. I was trying to use wireshark to capture packets and in such a way to see the communication. Here what I was doing:</p>\r\r\r\r\n\r\r\r\r\n<p>Pinging chesscube.com to realize its ip-address: 78.47.2.115\r\r\r\r\nthan I am listening o